In [13]:
## Text Mining Project - Experiments
# imports
import pandas as pd
import numpy as np
import re
import spacy
from gensim.corpora import Dictionary
from gensim.models import Phrases
from gensim.models import LdaModel
from gensim.models import nmf
from gensim.models import CoherenceModel
from operator import itemgetter
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from collections import Counter

In [4]:
# Save/Load DataFrame
#all_songs.to_pickle('all_songs.pkl')
all_songs = pd.read_pickle('all_songs.pkl')

In [5]:
# Preprocessing

nlp = spacy.load('de_core_news_lg')
nlp_en = spacy.load('en')
docs = []

print('Tokenizing and removing stop words')
# Tokenization and Lemmatization (remove stop words, numbers and one/two character words)
for index, song in all_songs.iterrows():
    # Remove the artist name from the lyrics
    doc = nlp(song['Lyrics'].replace(song['Artist'],''))
    lemmatized_doc = [token.lemma_ for token in doc if not token.is_stop and token.lemma_.isalpha() and len(token)>2]
    docs.append(lemmatized_doc)

# Remove english stop words and few manual stop words
manual_stop_words = ['ein', 'eine', 'mal', 'mach', 'lass', 'nich', 'sein', 'nix', 'tun', 'mein']
en_stop_words = nlp_en.Defaults.stop_words
docs = [[word for word in doc if not word.lower() in en_stop_words and not word.lower() in manual_stop_words] for doc in docs]

# Add bigrams/trigrams that appear 5 times or more (song texts are short)
bigram = Phrases(docs, min_count=5)
trigram = Phrases(bigram[docs], min_count=5)
print(f'Adding {len(bigram.vocab)} bigrams and {len(trigram.vocab)} trigrams')

for idx in range(len(docs)):
    for token in trigram[bigram[docs[idx]]]:
        if '_' in token:
            docs[idx].append(token)
    
# Filter extremes, these values seem to work well
dictionary = Dictionary(docs)
print(f'Filtering extremes - Number of unique tokens: {len(dictionary)}')
dictionary.filter_extremes(no_below=15, no_above=0.5)

print('Computing BoW representation')
# Bag-of-words representation of the documents
corpus = [dictionary.doc2bow(doc) for doc in docs]

temp = dictionary[0]
id2word = dictionary.id2token

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Tokenizing and removing stop words
Adding 936232 bigrams and 959381 trigrams
Filtering extremes - Number of unique tokens: 108140
Computing BoW representation
Number of unique tokens: 8195
Number of documents: 7467


In [9]:
def get_coherence_score(model):
    coherence_model = CoherenceModel(model=model, texts=docs, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    return coherence

In [44]:
# Find topic number - model combination with the highest coherence value

chunksize = 1000
passes = 100
iterations = 400
random_state = 18

model_results = {'Topics': [], 'Model': [], 'Coherence': []}
topics_range = range(3, 16)

for t in topics_range:  
        lda_model_ = LdaModel(
                 corpus=corpus,
                 id2word=id2word,
                 num_topics=t,
                 passes=passes,
                 iterations=iterations,
                 chunksize=chunksize,
                 random_state=random_state
        )
        cv = get_coherence_score(lda_model_)
        model_results['Model'].append('lda')
        model_results['Topics'].append(t)
        model_results['Coherence'].append(cv)
        print(f'model: lda, topics: {t}, score: {cv}')
        
for t in topics_range:  
        nmf_model_ = nmf.Nmf(
            corpus=corpus,
            num_topics=t,
            id2word=id2word,
            chunksize=chunksize,
            passes=passes,
            random_state=random_state
        )
        cv = get_coherence_score(nmf_model_)
        model_results['Model'].append('nmf')
        model_results['Topics'].append(t)
        model_results['Coherence'].append(cv)
        print(f'model: nmf, topics: {t}, score: {cv}')

model: lda, topics: 3, score: 0.3880800261358184
model: lda, topics: 4, score: 0.4139295223191907
model: lda, topics: 5, score: 0.4009439882922911
model: lda, topics: 6, score: 0.3912501411616684
model: lda, topics: 7, score: 0.40279858423161446
model: lda, topics: 8, score: 0.3727735490247208
model: lda, topics: 9, score: 0.4133459000515158
model: lda, topics: 10, score: 0.40702006361400594
model: lda, topics: 11, score: 0.41092463308253085
model: lda, topics: 12, score: 0.3878621844172901
model: lda, topics: 13, score: 0.36643452285847683
model: lda, topics: 14, score: 0.3827028079063838
model: lda, topics: 15, score: 0.3837642692500546
model: nmf, topics: 3, score: 0.35425503951984044
model: nmf, topics: 4, score: 0.3750091723051591
model: nmf, topics: 5, score: 0.36665627194491324
model: nmf, topics: 6, score: 0.36812054222826185
model: nmf, topics: 7, score: 0.4028816279027961
model: nmf, topics: 8, score: 0.33994943657547405
model: nmf, topics: 9, score: 0.38773649432746266
model

In [45]:
index, element = max(enumerate(model_results['Coherence']), key=itemgetter(1))

print(model_results['Topics'][index])
print(model_results['Model'][index])
print(model_results['Coherence'][index])

4
lda
0.4139295223191907


In [50]:
# Try LDA with 4 topics

random_state = 18
num_topics = 4
chunksize = 1000
passes = 250
iterations = 400

lda_model = LdaModel(
         corpus=corpus,
         id2word=id2word,
         num_topics=num_topics,
         passes=passes,
         iterations=iterations,
         chunksize=chunksize,
         random_state=random_state
)

print(f'{num_topics} Topics:')
print(get_coherence_score(lda_model))
lda_model.print_topics(num_words=30)

4 Topics:
0.4120638799712101


[(0,
  '0.012*"Bruder" + 0.010*"Geld" + 0.007*"Straße" + 0.006*"Bra" + 0.005*"Benz" + 0.005*"Jungs" + 0.005*"geben" + 0.005*"Gang" + 0.005*"Scheine" + 0.005*"Para" + 0.004*"Kopf" + 0.004*"Block" + 0.004*"Bratan" + 0.004*"weg" + 0.004*"Mama" + 0.004*"Fick" + 0.004*"Digga" + 0.004*"voll" + 0.004*"Gib" + 0.003*"Haze" + 0.003*"komm" + 0.003*"paar" + 0.003*"fick" + 0.003*"laufen" + 0.003*"Knast" + 0.003*"rein" + 0.003*"reden" + 0.003*"Bulle" + 0.003*"zieh" + 0.003*"schnellen"'),
 (1,
  '0.034*"Baby" + 0.016*"Bitch" + 0.016*"Dicka" + 0.015*"yeah" + 0.010*"weiß" + 0.009*"komm" + 0.009*"Bitches" + 0.009*"Money" + 0.007*"Geld" + 0.007*"Yeah" + 0.006*"Komm" + 0.005*"sag" + 0.005*"high" + 0.005*"Nacht" + 0.005*"bitte" + 0.005*"High" + 0.004*"Babe" + 0.004*"Ice" + 0.004*"jaja" + 0.004*"Club" + 0.004*"Party" + 0.004*"wissen" + 0.004*"geben" + 0.004*"Stay" + 0.004*"Gucci" + 0.004*"Boy" + 0.004*"Guck" + 0.003*"lieben" + 0.003*"Cash" + 0.003*"fühlen"'),
 (2,
  '0.011*"Leben" + 0.008*"Welt" + 0.007*"we

In [64]:
# Not implemented anymore
#performance_test_known(lda_model)

Predicted 55 of 77 (71%)
Incorrect labeling pairs: Counter({(2.0, 1): 6, (0.0, 1): 4, (2.0, 3): 3, (1.0, 2): 2, (3.0, 1): 2, (3.0, 0): 2, (0.0, 3): 2, (2.0, 0): 1})
Incorrect predictions: Counter({2.0: 10, 0.0: 6, 3.0: 4, 1.0: 2})
Incorrectly predicted labels: Counter({1: 12, 3: 5, 0: 3, 2: 2})


In [51]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
LDAvis_prepared

/home/luca/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.152973  0.115832       1        1  40.442348
3     -0.066891  0.071208       2        1  32.636963
0     -0.054132 -0.224701       3        1  15.733792
1      0.273996  0.037660       4        1  11.186891, topic_info=     Category         Freq     Term        Total  loglift  logprob
term                                                              
141   Default  3967.000000     Baby  3967.000000  30.0000  30.0000
437   Default  3327.000000    Bitch  3327.000000  29.0000  29.0000
671   Default  2783.000000     yeah  2783.000000  28.0000  28.0000
744   Default  1928.000000    Dicka  1928.000000  27.0000  27.0000
323   Default  3265.000000   Bruder  3265.000000  26.0000  26.0000
615   Default  3262.000000   Rapper  3262.000000  25.0000  25.0000
372   Default  2796.000000      Rap  2796.000000  24.0000  24.0000
430   Default  4826.000000     weiß  4826.000000  23.0000  23.0000
230   Default  4449.000000     Geld  4449.000000  22.0000  22.0000
629   Default  2554.000000     Yeah  2554.000000  21.0000  21.0000
1516  Default  1597.000000  Bitches  1597.000000  20.0000  20.0000
1182  Default  3217.000000   lieben  3217.000000  19.0000  19.0000
107   Default  4559.000000     komm  4559.000000  18.0000  18.0000
1882  Default  1203.000000    Money  1203.000000  17.0000  17.0000
161   Default  5493.000000    Leben  5493.000000  16.0000  16.0000
4094  Default   914.000000      Bra   914.000000  15.0000  15.0000
66    Default  2610.000000   Straße  2610.000000  14.0000  14.0000
48    Default  2673.000000    Nacht  2673.000000  13.0000  13.0000
159   Default  2330.000000     Komm  2330.000000  12.0000  12.0000
218   Default  1359.000000     Benz  1359.000000  11.0000  11.0000
80    Default  1679.000000    bitte  1679.000000  10.0000  10.0000
53    Default   743.000000     Para   743.000000   9.0000   9.0000
604   Default  1912.000000     Herz  1912.000000   8.0000   8.0000
23    Default  1991.000000    Jungs  1991.000000   7.0000   7.0000
264   Default  3634.000000     Welt  3634.000000   6.0000   6.0000
1051  Default  1417.000000    Block  1417.000000   5.0000   5.0000
4095  Default   676.000000   Bratan   676.000000   4.0000   4.0000
360   Default  1517.000000   Mensch  1517.000000   3.0000   3.0000
774   Default  2135.000000    Junge  2135.000000   2.0000   2.0000
280   Default   832.000000     high   832.000000   1.0000   1.0000
...       ...          ...      ...          ...      ...      ...
744    Topic4  1853.247803    Dicka  1928.671509   2.1505  -4.1420
1434   Topic4   531.838623     High   560.804138   2.1374  -5.3903
1882   Topic4  1023.855591    Money  1203.510986   2.0288  -4.7353
5358   Topic4   318.047211      Ufo   341.895721   2.1181  -5.9045
6185   Topic4   486.604004     jaja   554.322083   2.0601  -5.4792
3478   Topic4   360.477264    Louis   411.085052   2.0591  -5.7792
5909   Topic4   248.810059    ready   267.866699   2.1166  -6.1500
671    Topic4  1731.894897     yeah  2783.820312   1.7158  -4.2097
1516   Topic4  1065.969482  Bitches  1597.330688   1.7860  -4.6950
1767   Topic4   302.251892  Diamant   341.051147   2.0697  -5.9554
437    Topic4  1874.281860    Bitch  3327.071777   1.6166  -4.1307
280    Topic4   561.068726     high   832.687866   1.7956  -5.3368
3473   Topic4   426.085663    Gucci   590.942139   1.8633  -5.6120
2395   Topic4   416.195129      Boy   636.322327   1.7659  -5.6355
629    Topic4   838.728821     Yeah  2554.826904   1.0766  -4.9348
107    Topic4  1092.206909     komm  4559.493164   0.7614  -4.6707
430    Topic4  1113.937988     weiß  4826.950195   0.7241  -4.6510
159    Topic4   703.962463     Komm  2330.666992   0.9932  -5.1099
230    Topic4   848.868347     Geld  4449.586914   0.5338  -4.9228
1204   Topic4   471.380432     Club  1150.526855   1.2981  -5.5110
80     Topic4   535.687439    bitte  1679.664062   1.0476  -5.3831
249    Topic4  

In [54]:
# Try NMF with 4 topics as comparison, although coherence scores are lower for NMF models
# The topics don't make as much sense as with the LDA model -> stick to LDA

random_state = 18
num_topics = 4
chunksize = 1000
passes = 250

nmf_model = nmf.Nmf(
            corpus=corpus,
            num_topics=num_topics,
            id2word=id2word,
            chunksize=chunksize,
            passes=passes,
            random_state=random_state
        )

print(f'{num_topics} Topics:')
print(get_coherence_score(nmf_model))
nmf_model.print_topics(num_words=30)

4 Topics:
0.37832338288542194


[(0,
  '0.035*"yeah" + 0.026*"Geld" + 0.021*"Bitch" + 0.014*"Yeah" + 0.011*"lieben" + 0.008*"Bra" + 0.006*"komm" + 0.006*"fick" + 0.005*"Guck" + 0.005*"Fick" + 0.005*"Gang" + 0.005*"Ice" + 0.005*"Cash" + 0.004*"Rapper" + 0.004*"Boss" + 0.003*"Money" + 0.003*"Nutte" + 0.003*"Benz" + 0.003*"Komm" + 0.003*"Rap" + 0.003*"Block" + 0.003*"Mutter" + 0.003*"voll" + 0.003*"hol" + 0.003*"Louis" + 0.003*"paar" + 0.002*"flex" + 0.002*"geb" + 0.002*"guck" + 0.002*"drip"'),
 (1,
  '0.009*"Leben" + 0.006*"sehen" + 0.005*"bleiben" + 0.005*"Bruder" + 0.005*"zieh" + 0.004*"Welt" + 0.004*"Kopf" + 0.004*"Rapper" + 0.004*"raus" + 0.004*"Straße" + 0.004*"Rap" + 0.004*"Weg" + 0.004*"Junge" + 0.003*"weg" + 0.003*"stehen" + 0.003*"Mann" + 0.003*"sagen" + 0.003*"einfach" + 0.003*"denken" + 0.003*"geben" + 0.003*"Hand" + 0.003*"hören" + 0.003*"geh" + 0.003*"Freund" + 0.003*"leben" + 0.003*"reden" + 0.003*"Leute" + 0.003*"voll" + 0.003*"hart" + 0.003*"komm"'),
 (2,
  '0.190*"Bitches" + 0.065*"Money" + 0.042*"Gucc

In [46]:
# 4 Topics look promising, but better seperate the lifestyle topic "Gucci, Money, Geld" and the sex/party topic "Baby, Nacht, Party"
# Fine tune the model parameters eta and alpha for LDA with 5 topics

model_results = {'Alpha': [], 'Eta': [], 'Coherence': []}
alpha_values = [0.1, 0.5, 1, 5, 10, 'auto']
eta_values = [0.1, 0.5, 1, 5, 10, 'auto']

for alpha in alpha_values: 
    for eta in eta_values:
        lda_model_ = LdaModel(
                 corpus=corpus,
                 id2word=id2word,
                 num_topics=t,
                 passes=passes,
                 iterations=iterations,
                 alpha=alpha,
                 eta=eta,
                 chunksize=chunksize,
                 random_state=random_state
        )
        cv = get_coherence_score(lda_model_)
        model_results['Alpha'].append(alpha)
        model_results['Eta'].append(eta)
        model_results['Coherence'].append(cv)
        print(f'Alpha: {alpha}, Eta: {eta}, score: {cv}')

Alpha: 0.1, Eta: 0.1, score: 0.3715038624056148
Alpha: 0.1, Eta: 0.5, score: 0.41660750553151044
Alpha: 0.1, Eta: 1, score: 0.42807141609854304
Alpha: 0.1, Eta: 5, score: 0.4844000999801811
Alpha: 0.1, Eta: 10, score: 0.44727222519292137
Alpha: 0.1, Eta: auto, score: 0.3845428341052875
Alpha: 0.5, Eta: 0.1, score: 0.40070097904363977
Alpha: 0.5, Eta: 0.5, score: 0.4204101550425724
Alpha: 0.5, Eta: 1, score: 0.4155338746080693
Alpha: 0.5, Eta: 5, score: 0.4776549478330427
Alpha: 0.5, Eta: 10, score: 0.4644459908352593
Alpha: 0.5, Eta: auto, score: 0.40000797788204945
Alpha: 1, Eta: 0.1, score: 0.4271346844149047
Alpha: 1, Eta: 0.5, score: 0.4456970870343123
Alpha: 1, Eta: 1, score: 0.4251672692223079
Alpha: 1, Eta: 5, score: 0.4814976034523775
Alpha: 1, Eta: 10, score: 0.4086932751398091
Alpha: 1, Eta: auto, score: 0.43011616857432206
Alpha: 5, Eta: 0.1, score: 0.4193349867731637
Alpha: 5, Eta: 0.5, score: 0.4294292099922394
Alpha: 5, Eta: 1, score: 0.436802134968805
Alpha: 5, Eta: 5, s

In [47]:
index, element = max(enumerate(model_results['Coherence']), key=itemgetter(1))

print(model_results['Alpha'][index])
print(model_results['Eta'][index])
print(model_results['Coherence'][index])

auto
10
0.4859971551318659


In [48]:
# Try LDA 5 topics with eta=10 and alpha='auto'

random_state = 18
num_topics = 5
chunksize = 1000
passes = 250
iterations = 10000

lda_model = LdaModel(
         corpus=corpus,
         id2word=id2word,
         num_topics=num_topics,
         passes=passes,
         iterations=iterations,
         chunksize=chunksize,
         alpha='auto',
         eta=10,
         random_state=random_state
)

print(f'{num_topics} Topics:')
print(get_coherence_score(lda_model))
lda_model.print_topics(num_words=30)

5 Topics:
0.46126473109769084


[(0,
  '0.002*"mhm" + 0.002*"Tanz" + 0.001*"Mhm" + 0.001*"Vulkan" + 0.001*"Raver" + 0.001*"Fahrer" + 0.001*"Chemie" + 0.001*"Salat" + 0.001*"Bello" + 0.000*"Feuerzeug" + 0.000*"Water" + 0.000*"tanz" + 0.000*"bad" + 0.000*"Finale" + 0.000*"Harz" + 0.000*"lala" + 0.000*"vergiften" + 0.000*"Achtung" + 0.000*"Chuck" + 0.000*"reiten" + 0.000*"Fahne" + 0.000*"released" + 0.000*"song" + 0.000*"General" + 0.000*"Walkman" + 0.000*"Hitler" + 0.000*"Kugelhagel" + 0.000*"Mademoiselle" + 0.000*"Arbeit" + 0.000*"ansprechen"'),
 (1,
  '0.003*"like" + 0.002*"Lachs" + 0.002*"ready" + 0.002*"love" + 0.002*"Harry" + 0.002*"Potter" + 0.002*"que" + 0.002*"know" + 0.001*"pas" + 0.001*"heavy" + 0.001*"come" + 0.001*"Bounce" + 0.001*"away" + 0.001*"need" + 0.001*"les" + 0.001*"let" + 0.001*"good" + 0.001*"wanna" + 0.001*"nick" + 0.001*"want" + 0.001*"Champion" + 0.001*"got" + 0.001*"run" + 0.001*"eyes" + 0.001*"Dragon" + 0.001*"Ick" + 0.001*"ick" + 0.001*"believe" + 0.001*"Ferris" + 0.001*"think"'),
 (2,
  '0

In [49]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
LDAvis_prepared

/home/luca/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.105565  0.039838       1        1  48.117645
3      0.043572 -0.005682       2        1  31.078438
4      0.008420 -0.076267       3        1  19.487022
1     -0.080894  0.020677       4        1   1.035467
0     -0.076663  0.021434       5        1   0.281428, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
744   Default  1471.000000         Dicka  1471.000000  30.0000  30.0000
615   Default  2584.000000        Rapper  2584.000000  29.0000  29.0000
141   Default  3589.000000          Baby  3589.000000  28.0000  28.0000
437   Default  2769.000000         Bitch  2769.000000  27.0000  27.0000
372   Default  2221.000000           Rap  2221.000000  26.0000  26.0000
1882  Default   938.000000         Money   938.000000  25.0000  25.0000
230   Default  3575.000000          Geld  3575.000000  24.0000  24.0000
1516  Default  1284.000000       Bitches  1284.000000  23.0000  23.0000
1182  Default  2803.000000        lieben  2803.000000  22.0000  22.0000
4094  Default   669.000000           Bra   669.000000  21.0000  21.0000
95    Default  1794.000000          fick  1794.000000  20.0000  20.0000
671   Default  2300.000000          yeah  2300.000000  19.0000  19.0000
161   Default  4652.000000         Leben  4652.000000  18.0000  18.0000
323   Default  2680.000000        Bruder  2680.000000  17.0000  17.0000
227   Default  1157.000000          Gang  1157.000000  16.0000  16.0000
4095  Default   498.000000        Bratan   498.000000  15.0000  15.0000
774   Default  1742.000000         Junge  1742.000000  14.0000  14.0000
448   Default  1047.000000          King  1047.000000  13.0000  13.0000
604   Default  1654.000000          Herz  1654.000000  12.0000  12.0000
264   Default  3093.000000          Welt  3093.000000  11.0000  11.0000
53    Default   563.000000          Para   563.000000  10.0000  10.0000
149   Default   913.000000          Boss   913.000000   9.0000   9.0000
218   Default  1078.000000          Benz  1078.000000   8.0000   8.0000
1530  Default   998.000000         Nutte   998.000000   7.0000   7.0000
225   Default  1562.000000          Fick  1562.000000   6.0000   6.0000
3473  Default   466.000000         Gucci   466.000000   5.0000   5.0000
610   Default   711.000000           Mic   711.000000   4.0000   4.0000
1156  Default   911.000000       Schwanz   911.000000   3.0000   3.0000
2471  Default   141.000000          Tanz   141.000000   2.0000   2.0000
5     Default   843.000000          Cash   843.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
7996   Topic5     1.399434     Feuerzeug    42.214176   2.4664  -7.6480
4457   Topic5     1.935714         Salat    62.403511   2.3999  -7.3236
5388   Topic5     1.377764         Water    45.946606   2.3660  -7.6636
7376   Topic5     1.174614        Finale    41.923500   2.2981  -7.8231
5221   Topic5     0.972959         Chuck    38.528179   2.1942  -8.0115
1134   Topic5     1.110063          Harz    44.413750   2.1839  -7.8797
6209   Topic5     0.891585      released    36.531548   2.1601  -8.0988
6307   Topic5     0.891577          song    38.269405   2.1136  -8.0988
6564   Topic5     0.885697       General    42.876343   1.9933  -8.1054
7692   Topic5     0.776301  Mademoiselle    39.588863   1.9413  -8.2373
7135   Topic5     1.065873          lala    60.605118   1.8325  -7.9203
7079   Topic5     0.663784       Chinese    38.728313   1.8067  -8.3939
6070   Topic5     1.256115           bad    75.339813   1.7791  -7.7560
6433   Topic5     0.792276    Kugelhagel    48.391109   1.7609  -8.2169
6414   Topic5     0.696115         wilde    42.793285   1.7544  -8.3463
7563   Topic5     0.623952      Dynastie    40.843975   1.6916  -8.4558
5495   Topic5     0.695379         Pokal    45.6002

In [59]:
# Really bad results, try 'auto' for parameter eta instead

random_state = 18
num_topics = 5
chunksize = 1000
passes = 250
iterations = 10000

lda_model = LdaModel(
         corpus=corpus,
         id2word=id2word,
         num_topics=num_topics,
         passes=passes,
         iterations=iterations,
         chunksize=chunksize,
         alpha='auto',
         eta='auto',
         random_state=random_state
)

print(f'{num_topics} Topics:')
print(get_coherence_score(lda_model))
lda_model.print_topics(num_words=30)

5 Topics:
0.4067005288169977


[(0,
  '0.020*"Bruder" + 0.015*"Geld" + 0.014*"Straße" + 0.011*"Mama" + 0.010*"Kopf" + 0.009*"Jungs" + 0.007*"raus" + 0.006*"Leben" + 0.006*"Block" + 0.006*"weg" + 0.006*"laufen" + 0.006*"Knast" + 0.005*"bleiben" + 0.005*"Stadt" + 0.005*"Weg" + 0.005*"rein" + 0.005*"reden" + 0.005*"leben" + 0.005*"Bulle" + 0.005*"schnellen" + 0.005*"Berlin" + 0.004*"Bratan" + 0.004*"Scheine" + 0.004*"Kugel" + 0.004*"paar" + 0.004*"voll" + 0.004*"Ghetto" + 0.004*"Feind" + 0.004*"geh" + 0.004*"Benz"'),
 (1,
  '0.041*"Baby" + 0.020*"komm" + 0.017*"weiß" + 0.014*"Komm" + 0.011*"yeah" + 0.011*"sag" + 0.010*"bitte" + 0.010*"Party" + 0.010*"zieh" + 0.009*"Nacht" + 0.008*"Club" + 0.007*"high" + 0.007*"geben" + 0.007*"voll" + 0.006*"tanzen" + 0.006*"wissen" + 0.005*"Babe" + 0.005*"Sex" + 0.005*"bleib" + 0.005*"Frau" + 0.005*"Yeah" + 0.005*"chill" + 0.005*"Nummer" + 0.005*"brauch" + 0.004*"hol" + 0.004*"vorbei" + 0.004*"Flasche" + 0.004*"Haus" + 0.004*"brauchen" + 0.004*"machen"'),
 (2,
  '0.013*"Leben" + 0.009*

In [60]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
LDAvis_prepared

/home/luca/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.200142  0.007702       1        1  33.710419
3      0.032789 -0.043587       2        1  29.122099
0      0.150395 -0.140771       3        1  15.446563
4     -0.315043 -0.134226       4        1  11.761920
1     -0.068283  0.310881       5        1   9.958997, topic_info=     Category         Freq      Term        Total  loglift  logprob
term                                                               
141   Default  4252.000000      Baby  4252.000000  30.0000  30.0000
323   Default  3294.000000    Bruder  3294.000000  29.0000  29.0000
430   Default  4850.000000      weiß  4850.000000  28.0000  28.0000
107   Default  4553.000000      komm  4553.000000  27.0000  27.0000
744   Default  1988.000000     Dicka  1988.000000  26.0000  26.0000
66    Default  2590.000000    Straße  2590.000000  25.0000  25.0000
437   Default  3421.000000     Bitch  3421.000000  24.0000  24.0000
161   Default  5484.000000     Leben  5484.000000  23.0000  23.0000
230   Default  4463.000000      Geld  4463.000000  22.0000  22.0000
1182  Default  3297.000000    lieben  3297.000000  21.0000  21.0000
615   Default  3253.000000    Rapper  3253.000000  20.0000  20.0000
372   Default  2784.000000       Rap  2784.000000  19.0000  19.0000
159   Default  2311.000000      Komm  2311.000000  18.0000  18.0000
359   Default  1792.000000      Mama  1792.000000  17.0000  17.0000
671   Default  2868.000000      yeah  2868.000000  16.0000  16.0000
204   Default  2823.000000       sag  2823.000000  15.0000  15.0000
80    Default  1666.000000     bitte  1666.000000  14.0000  14.0000
1882  Default  1232.000000     Money  1232.000000  13.0000  13.0000
1516  Default  1624.000000   Bitches  1624.000000  12.0000  12.0000
23    Default  1996.000000     Jungs  1996.000000  11.0000  11.0000
728   Default  1215.000000      zieh  1215.000000  10.0000  10.0000
48    Default  2695.000000     Nacht  2695.000000   9.0000   9.0000
249   Default  1123.000000     Party  1123.000000   8.0000   8.0000
36    Default  3012.000000      Kopf  3012.000000   7.0000   7.0000
264   Default  3643.000000      Welt  3643.000000   6.0000   6.0000
95    Default  2266.000000      fick  2266.000000   5.0000   5.0000
1204  Default  1136.000000      Club  1136.000000   4.0000   4.0000
604   Default  1923.000000      Herz  1923.000000   3.0000   3.0000
627   Default  2490.000000       Weg  2490.000000   2.0000   2.0000
629   Default  2603.000000      Yeah  2603.000000   1.0000   1.0000
...       ...          ...       ...          ...      ...      ...
1221   Topic5   257.332031     Level   284.322449   2.2070  -6.0000
375    Topic5   548.707275       Sex   693.684265   2.0722  -5.2428
1929   Topic5   457.532043   Flasche   569.794800   2.0873  -5.4246
1607   Topic5   373.689301     flieg   461.164276   2.0964  -5.6270
159    Topic5  1409.350952      Komm  2311.808350   1.8118  -4.2995
80     Topic5  1055.629517     bitte  1666.443359   1.8501  -4.5885
107    Topic5  2087.328125      komm  4553.346191   1.5267  -3.9068
1667   Topic5   422.312256   trinken   571.217834   2.0047  -5.5047
648    Topic5   285.878174      like   346.360260   2.1148  -5.8948
764    Topic5   384.498993      Glas   525.627747   1.9940  -5.5985
430    Topic5  1770.825195      weiß  4850.794922   1.2990  -4.0712
204    Topic5  1099.656006       sag  2823.964355   1.3635  -4.5477
671    Topic5  1104.447998      yeah  2868.451660   1.3523  -4.5433
773    Topic5   374.552734     Joint   521.880615   1.9750  -5.6247
48     Topic5   918.801025     Nacht  2695.189453   1.2305  -4.7273
69     Topic5   369.970734     Tisch   520.647217   1.9650  -5.6370
1150   Topic5   487.136841    Nummer   918.606934   1.6724  -5.3619
305    Topic5   675.631348      voll  2627.238281   0.9487  -5.0348
277    Topic5   691.140991     geben  3109.821777   0.8027  -5.0121
82     Topic5   533.767395     bleib  1591.